## Gradient Boosting

In GBDT, we are trying to construct a ensemble algorithm with multiple "weak algorithms". Let $F(X_i) = f_1(X_i) + ... + f_m(X_i)$, we are constructing a ensemble algorithm F(X_i) with the sum of weak algorithms $\{f_n(X_i)\}_{n=1,2,...}$, in GBDT, $f_n$ are choosen to be shallow decision trees.

When $Loss = \sum_i L(y_i,F(X_i))$, we want to minimize the Loss, however, we can not directly do gradient descent on parameters of $F(X_i)$, since it is consist of multiple decision trees, and the parameters of decision trees are just ```split_feature_index``` and ```split_threshold```, which we can not get their conresponding gradient. But since $F_{m}(X_i) = F_{m-1}(X_i)+f_m(X_i)$, we can consider the whole $F$ as a parameter, and Loss is only related to one Variable -- $F$, so we only need to do gradient descent of $Loss(F)$ on F:

We can use a normal function g(x) to help us understand this step better. If we want to get smaller g(x), we can update x as $x_t = x_{t-1}+\alpha\cdot(-\frac{\partial g}{\partial x_{t-1}})$. Similarily, For each sample $X_i$, we want to update  $F_{m}(X_i) = F_{m-1}(X_i)-\alpha_i\cdot\frac{\partial Loss}{\partial F_{m-1}(X_i)}$, where we can consider $residual_i = -\frac{\partial Loss}{\partial F_{m-1}(X_i)}$ as the "direction" to update F.

So, we can use $\{(X_i,residual_i)\}_{i=1,2...}$ as training data to fit a regression decision tree. So far, we only know how to get the "direction", and use $residual_i$ to fit a regression decision tree to simulate the "direction". But how about the "step size"? We can get step size in following steps:

$\alpha = argmin_{\alpha}\sum_i Loss(y_i,F_{m-1}(X_i) + \sum_j\alpha_j\cdot I(X_i\in R_{jm}))$, where $R_{m}$ is the regression decision tree fitted based on $\{(X_i,-\frac{\partial Loss}{\partial F_{m-1}(X_i)})\}$, and $R_{jm}$ is the $j^{th}$ leaf node of $R_m$. Here $\alpha$ can be estimated via one step newton-raphson (ref: https://stats.stackexchange.com/questions/330849/how-do-newton-raphson-updates-work-in-gradient-boosting):

$\alpha_j = -\frac{\sum_{i\in R_{mj}}\frac{\partial Loss}{\partial F_{m-1}(X_i)}}{\sum_{i\in R_{mj}}\frac{\partial ^2 Loss}{\partial  F_{m-1}(X_i)^2}}$

## Loss function

The Loss function we choose for binary classification here is Bionominal Loss (can also be regarded as maximum Loglikelihood):

Given $\{(y_i,p_i)\}$, where $y_i\in\{0,1\}$, $p_i$ is the probability of being label $y_i=1$, so $Loss = -\sum_i (y_i\cdot log p_i + (1-y_i)\cdot log (1-p_i))$. We choose $F$ to be logits of $p$, i.e. $F(X_i) = log \frac{p_i}{1-p_i}$, therefore, $Loss = -\sum_i [y_i\cdot F(X_i) - log (1+e^{F(X_i)})]$.

$\frac{\partial Loss}{\partial F(X_i)} = -y_i + \frac{e^{F(X_i)}}{1+e^{F(X_i)}}$, and according to previous content, we know that the $residual_i$ we used to fit new decision tree is $-\frac{\partial Loss}{\partial F(X_i)}$, so we have  $-y_i + \frac{e^{F(X_i)}}{1+e^{F(X_i)}} = - residual_i \Rightarrow \frac{e^{F(X_i)}}{1+e^{F(X_i)}} = y_i - residual_i$ 

$\frac{\partial ^2 Loss}{\partial  F_{m-1}(X_i)^2} = \frac{e^{F(X_i)}}{1+e^{F(X_i)}}\cdot(1-\frac{e^{F(X_i)}}{1+e^{F(X_i)}}) = (y_i-residual_i)\cdot(1-y_i+residual_i)$

